# Computing tiles on the fly from an Xarray dataset

In this notebook, we'll show how to visualize local data. To execute it, you will need to install these packages:

In [1]:
!pip install rioxarray xarray rich ipywidgets dask matplotlib

In [2]:
from pathlib import Path
from zipfile import ZipFile

import httpx
import rioxarray
import xarray as xr
from rich.progress import track

from jupytergis.tiler import GISDocument

We will display the DEM (digital elevation model) for Asia from the [HydroSHEDS](https://www.hydrosheds.org/) dataset, which represents the terrain. Let's first download the data:

In [3]:
url = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/as_dem_30s_grid.zip"
filename = Path(url).name
name = filename[: filename.find("_grid")]
adffile = Path(name) / name / "w001001.adf"

if not adffile.exists():
    with httpx.stream("GET", url) as r, open(filename, "wb") as f:
        total = int(r.headers["Content-Length"]) / 1024
        for data in track(
            r.iter_bytes(chunk_size=1024), total=total, description="Downloading"
        ):
            f.write(data)
        f.flush()
    zip = ZipFile(filename)
    zip.extractall(".")

It is a dataset that [Rasterio](https://rasterio.readthedocs.io) can open, but in order to get a nice `DataArray` with all the metadata taken into account, we'll open it with [rioxarray](https://corteva.github.io/rioxarray):

In [4]:
da = rioxarray.open_rasterio(adffile, masked=True)
da

<xarray.DataArray (band: 1, y: 8400, x: 15000)> Size: 504MB
[126000000 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 120kB 55.0 55.01 55.02 55.03 ... 180.0 180.0 180.0
  * y            (y) float64 67kB 60.0 59.99 59.98 ... -9.979 -9.988 -9.996
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

The projection is `EPSG:4326` (aka `WGS84`). Here the coordinate `x` corresponds to longitudes, and `y` to latitudes (in degrees). There is only one band.

In [5]:
da = da.sel(band=1)

The dataset can be too big to hold in memory, so we will chunk it into smaller pieces. That will also improve performances as the generation of a tile can be done in parallel using Dask.

In [6]:
da = da.chunk(dict(x=1000, y=1000))

Now let's save this data array in a dataset, as a Zarr store:

We'll need the minimum and maximum values of the DEM to rescale our tiles:

In [7]:
vmin, vmax = int(da.min().compute()), int(da.max().compute())
vmin, vmax

(-158, 8247)

We are now ready to show a *tiler layer*, which computes the tiles on the fly:

In [8]:
Path("xarray.jGIS").unlink(missing_ok=True)
doc = GISDocument("xarray.jGIS")
doc

In [ ]:
await doc.add_tiler_layer(
    name="Xarray Layer",
    data_array=da,
    colormap_name="viridis",
    rescale=(vmin, vmax),
)

In [ ]:
# not mandatory, but helps shutting down the kernel gracefully:
# await doc.stop_tile_server()